<a href="https://colab.research.google.com/github/RyukiMizo/YAD2K/blob/master/yolov2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
from google.colab import drive
drive.mount('/content/drive')

AttributeError: ignored

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
!cp MAP.py drive/'My Drive'/'Colab Notebooks'/

In [16]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
%matplotlib inline

Using TensorFlow backend.


<h2>Filtering with a threshold on class  scores</h2>

In [0]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = .6):
    box_scores = np.multiply(box_confidence, box_class_probs)

    box_classes = K.argmax(box_scores, axis = -1)#indexの所得
    #1のindex 19*19*5ある -1ってことは要は80に対して行なっていると。
    box_class_scores = K.max(box_scores, axis = -1)
    filtering_mask = box_class_scores >= threshold

    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    boxes = tf.boolean_mask(boxes, filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)
  
    return scores, boxes, classes

<h2>Non-max suppression</h2>

In [0]:
def iou(pred_bb, true_bb):
    pred_area = (pred_bb[2]-pred_bb[0])*(pred_bb[3]-pred_bb[1])
    true_area = (true_bb[2]-true_bb[0])*(true_bb[3]-true_bb[1])
    intersection_x = max(min(pred_bb[2],true_bb[2])-max(pred_bb[0],true_bb[0]),0)
    intersection_y = max(min(pred_bb[3],true_bb[3])-max(pred_bb[1],true_bb[1]),0)
    intersection_area = intersection_x*intersection_y
    union_area = pred_area+true_area-intersection_area
    if union_area>0:
        return intersection_area/union_area
    else:
        return 0

In [0]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = .7):
    
    max_boxes_tensor = K.variable(max_boxes, dtype='int32')     # tensor to be used in tf.image.non_max_suppression()
    K.get_session().run(tf.variables_initializer([max_boxes_tensor])) # initialize variable max_boxes_tensor
    nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes, iou_threshold)
    scores = K.gather(scores, nms_indices)
    boxes =  K.gather(boxes, nms_indices)
    classes =  K.gather(classes, nms_indices)
    
    return scores, boxes, classes

In [0]:
def yolo_eval(yolo_outputs, image_shape = (1216., 1936.), max_boxes=10, score_threshold=.6, iou_threshold=.7):

    box_xy, box_wh, box_confidence, box_class_probs = yolo_outputs

    # Convert boxes to be ready for filtering functions 
    boxes = yolo_boxes_to_corners(box_xy, box_wh)

    # Use one of the functions you've implemented to perform Score-filtering with a threshold of score_threshold (≈1 line)
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = score_threshold)
    
    # Scale boxes back to original image shape.
    boxes = scale_boxes(boxes, image_shape)

    # Use one of the functions you've implemented to perform Non-max suppression with a threshold of iou_threshold (≈1 line)
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = iou_threshold)
 
    
    return scores, boxes, classes

In [0]:
sess = K.get_session()

In [47]:
%%writefile model_data/coco_classes.txt
Pedestrian
Bicycle
Car
motorbike
aeroplane
bus
train
Truck
boat
Signal
fire hydrant
Sign
parking meter
bench
bird
cat
dog
horse
sheep
cow
elephant
bear
zebra
giraffe
backpack
umbrella
handbag
tie
suitcase
frisbee
skis
snowboard
sports ball
kite
baseball bat
baseball glove
skateboard
surfboard
tennis racket
bottle
wine glass
cup
fork
knife
spoon
bowl
banana
apple
sandwich
orange
broccoli
carrot
hot dog
pizza
donut
cake
chair
sofa
pottedplant
bed
diningtable
toilet
tvmonitor
laptop
mouse
remote
keyboard
cell phone
microwave
oven
toaster
sink
refrigerator
book
clock
vase
scissors
teddy bear
hair drier
toothbrush

Overwriting model_data/coco_classes.txt


In [0]:
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")
image_shape = (1216., 1936.)

In [11]:
!git clone https://github.com/allanzelener/YAD2K.git
%cd {'YAD2K'}
!wget http://pjreddie.com/media/files/yolo.weights
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov2.cfg
!python ./yad2k.py yolov2.cfg yolo.weights model_data/yolo.h5
yolo_model = load_model("model_data/yolo.h5")

Cloning into 'YAD2K'...
remote: Enumerating objects: 243, done.
remote: Total 243 (delta 0), reused 0 (delta 0), pack-reused 243
Receiving objects: 100% (243/243), 2.35 MiB | 24.07 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [24]:
yolo_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 608, 608, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 608, 608, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 608, 608, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 608, 608, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [0]:
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

In [0]:
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

In [0]:
scores, boxes, classes = yolo_eval(yolo_outputs, image_shape)

In [0]:
def predict(sess, image_file):
    """
    Runs the graph stored in "sess" to predict boxes for "image_file". Prints and plots the preditions.
    
    Arguments:
    sess -- your tensorflow/Keras session containing the YOLO graph
    image_file -- name of an image stored in the "images" folder.
    
    Returns:
    out_scores -- tensor of shape (None, ), scores of the predicted boxes
    out_boxes -- tensor of shape (None, 4), coordinates of the predicted boxes
    out_classes -- tensor of shape (None, ), class index of the predicted boxes
    
    Note: "None" actually represents the number of predicted boxes, it varies between 0 and max_boxes. 
    """

    # Preprocess your image
    image, image_data = preprocess_image("images/" + image_file, model_image_size = (608, 608))

    # Run the session with the correct tensors and choose the correct placeholders in the feed_dict.
    # You'll need to use feed_dict={yolo_model.input: ... , K.learning_phase(): 0})
    ### START CODE HERE ### (≈ 1 line)
    out_scores, out_boxes, out_classes = sess.run(yolo_eval(yolo_outputs, image_shape),
                                                  feed_dict={yolo_model.input:image_data,K.learning_phase():0})
    yolo_eval(yolo_outputs, image_shape = (720., 1280.), max_boxes=10, score_threshold=.6, iou_threshold=.5)
    ### END CODE HERE ###

    # Print predictions info
    print('Found {} boxes for {}'.format(len(out_boxes), image_file))
    # Generate colors for drawing bounding boxes.
    colors = generate_colors(class_names)
    # Draw bounding boxes on the image file
    draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
    # Save the predicted bounding box on the image
    image.save(os.path.join("out", image_file), quality=90)
    # Display the results in the notebook
    output_image = scipy.misc.imread(os.path.join("out", image_file))
    imshow(output_image)
    
    return out_scores, out_boxes, out_classes

In [9]:
{'test_0000.jpg': {'Bicycle': [[0, 0, 1936, 1216]],
  'Car': [[0, 0, 1936, 1216]],
  'Pedestrian': [[0, 0, 1936, 1216]],
  'Signal': [[0, 0, 1936, 1216]],
  'Signs': [[0, 0, 1936, 1216]],
  'Truck': [[0, 0, 1936, 1216]]},
 'test_0001.jpg': {'Bicycle': [[0, 0, 1936, 1216]],
  'Car': [[0, 0, 1936, 1216]],
  'Pedestrian': [[0, 0, 1936, 1216]],
  'Signal': [[0, 0, 1936, 1216]],
  'Signs': [[0, 0, 1936, 1216]],
  'Truck': [[0, 0, 1936, 1216]]},
 'test_0002.jpg': {'Bicycle': [[0, 0, 1936, 1216]],
  'Car': [[0, 0, 1936, 1216]],
  'Pedestrian': [[0, 0, 1936, 1216]],
  'Signal': [[0, 0, 1936, 1216]],

{'test_0000.jpg': {'Bicycle': [[0, 0, 1936, 1216]],
  'Car': [[0, 0, 1936, 1216]],
  'Pedestrian': [[0, 0, 1936, 1216]],
  'Signal': [[0, 0, 1936, 1216]],
  'Signs': [[0, 0, 1936, 1216]],
  'Truck': [[0, 0, 1936, 1216]]},
 'test_0001.jpg': {'Bicycle': [[0, 0, 1936, 1216]],
  'Car': [[0, 0, 1936, 1216]],
  'Pedestrian': [[0, 0, 1936, 1216]],
  'Signal': [[0, 0, 1936, 1216]],
  'Signs': [[0, 0, 1936, 1216]],
  'Truck': [[0, 0, 1936, 1216]]},
 'test_0002.jpg': {'Bicycle': [[0, 0, 1936, 1216]],
  'Car': [[0, 0, 1936, 1216]],
  'Pedestrian': [[0, 0, 1936, 1216]],
  'Signal': [[0, 0, 1936, 1216]],
  'Signs': [[0, 0, 1936, 1216]],
  'Truck': [[0, 0, 1936, 1216]]},
 'test_0003.jpg': {'Bicycle': [[0, 0, 1936, 1216]],
  'Car': [[0, 0, 1936, 1216]],
  'Pedestrian': [[0, 0, 1936, 1216]],
  'Signal': [[0, 0, 1936, 1216]],
  'Signs': [[0, 0, 1936, 1216]],
  'Truck': [[0, 0, 1936, 1216]]},
 'test_0004.jpg': {'Bicycle': [[0, 0, 1936, 1216]],
  'Car': [[0, 0, 1936, 1216]],
  'Pedestrian': [[0, 0, 1936,

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Mon Nov 12 23:09:30 2018

@author: n.aoi
"""

import json
import argparse

def MMAP(true, pred, threshold):
    eval_files = set(true).intersection(set(pred))
    score = 0
    count = 0
    for eval_file in eval_files:
        score += MAP(pred[eval_file], true[eval_file], threshold)
        count+=1
    return score/len(true)

def MAP(pr, gt, threshold, max_target_labels = 100):
    num_true_categories = len(gt)
    aps = 0
    for c, pred_bbs in pr.items():
        ap = 0
        detected = 0
        if c not in gt:
            num_true = 0
        else:
            num_true = len(gt[c])
        
        if num_true>0:
            del_recall = 1/min(num_true, max_target_labels)
            for i,pred_bb in enumerate(pred_bbs[:max_target_labels]):
                if len(gt[c])>0:
                    pred_true_dist = {j:compute_iou_bb(pred_bb, true_bb) for j,true_bb in enumerate(gt[c])}
                    nearest = max(pred_true_dist.items(), key = lambda x:x[1])
                    if nearest[1]>=threshold:
                        detected+=1
                        ap += detected/(i+1)
                        del gt[c][nearest[0]]
                else:
                    break
            ap *= del_recall
        aps+=ap
    return aps/num_true_categories

def compute_iou_bb(pred_bb, true_bb):
    pred_area = (pred_bb[2]-pred_bb[0])*(pred_bb[3]-pred_bb[1])
    true_area = (true_bb[2]-true_bb[0])*(true_bb[3]-true_bb[1])
    intersection_x = max(min(pred_bb[2],true_bb[2])-max(pred_bb[0],true_bb[0]),0)
    intersection_y = max(min(pred_bb[3],true_bb[3])-max(pred_bb[1],true_bb[1]),0)
    intersection_area = intersection_x*intersection_y
    union_area = pred_area+true_area-intersection_area
    if union_area>0:
        return intersection_area/union_area
    else:
        return 0

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('-g', '--path_to_ground_truth', type = str, help = 'path to ground truth', nargs='?')
    parser.add_argument('-p', '--path_to_prediction', type = str, help = 'path to prediction', nargs='?')
    parser.add_argument('-t', '--threshold', type = float, help = 'threshold', default = 0.75, nargs='?')
    args = parser.parse_args()
    
    with open(args.path_to_ground_truth) as f:
        true = json.load(f)
    with open(args.path_to_prediction) as f:
        pred = json.load(f)
    
    score = MMAP(true, pred, args.threshold)
    print(score)

In [0]:
import colorsys
import imghdr
import os
import random
from keras import backend as K

import numpy as np
from PIL import Image, ImageDraw, ImageFont

def read_classes(classes_path):
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def read_anchors(anchors_path):
    with open(anchors_path) as f:
        anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        anchors = np.array(anchors).reshape(-1, 2)
    return anchors

def generate_colors(class_names):
    hsv_tuples = [(x / len(class_names), 1., 1.) for x in range(len(class_names))]
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), colors))
    random.seed(10101)  # Fixed seed for consistent colors across runs.
    random.shuffle(colors)  # Shuffle colors to decorrelate adjacent classes.
    random.seed(None)  # Reset seed to default.
    return colors

def scale_boxes(boxes, image_shape):
    """ Scales the predicted boxes in order to be drawable on the image"""
    height = image_shape[0]
    width = image_shape[1]
    image_dims = K.stack([height, width, height, width])
    image_dims = K.reshape(image_dims, [1, 4])
    boxes = boxes * image_dims
    return boxes

def preprocess_image(img_path, model_image_size):
    image_type = imghdr.what(img_path)
    image = Image.open(img_path)
    resized_image = image.resize(tuple(reversed(model_image_size)), Image.BICUBIC)
    image_data = np.array(resized_image, dtype='float32')
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0)  # Add batch dimension.
    return image, image_data

def draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors):
    
    font = ImageFont.truetype(font='font/FiraMono-Medium.otf',size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
    thickness = (image.size[0] + image.size[1]) // 300

    for i, c in reversed(list(enumerate(out_classes))):
        predicted_class = class_names[c]
        box = out_boxes[i]
        score = out_scores[i]

        label = '{} {:.2f}'.format(predicted_class, score)

        draw = ImageDraw.Draw(image)
        label_size = draw.textsize(label, font)

        top, left, bottom, right = box
        top = max(0, np.floor(top + 0.5).astype('int32'))
        left = max(0, np.floor(left + 0.5).astype('int32'))
        bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
        right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
        print(label, (left, top), (right, bottom))

        if top - label_size[1] >= 0:
            text_origin = np.array([left, top - label_size[1]])
        else:
            text_origin = np.array([left, top + 1])

        # My kingdom for a good redistributable image drawing library.
        for i in range(thickness):
            draw.rectangle([left + i, top + i, right - i, bottom - i], outline=colors[c])
        draw.rectangle([tuple(text_origin), tuple(text_origin + label_size)], fill=colors[c])
        draw.text(text_origin, label, fill=(0, 0, 0), font=font)
        del draw

In [71]:
!unzip -q drive/'My Drive'/'Colab Notebooks'/.zip

(<tf.Tensor 'truediv_10:0' shape=(?, ?, ?, 5, 2) dtype=float32>,
 <tf.Tensor 'truediv_11:0' shape=(?, ?, ?, 5, 2) dtype=float32>,
 <tf.Tensor 'Sigmoid_3:0' shape=(?, ?, ?, 5, 1) dtype=float32>,
 <tf.Tensor 'Softmax_1:0' shape=(?, ?, ?, 5, 80) dtype=float32>)

In [0]:
a = tf.random_normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1)
b = tf.random_normal([19, 19, 5, 4], mean=1, stddev=4, seed = 1)

In [58]:
a*b

<tf.Tensor 'mul_5:0' shape=(19, 19, 5, 4) dtype=float32>